In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.decomposition import PCA

### reading directory

In [2]:
# dataset that contains images' path and label
data = pd.DataFrame()
data['image-name'] = os.listdir('../data/flowers/')

# extracting species
data['species'] = data['image-name'].apply(lambda x: int(x[:2]))

# saving "true" label
np.save('../data/true_label.npy', data['species'].values)

### raw images

In [3]:
raw_images_flatten = np.array([plt.imread('../data/flowers/'+img).ravel() for img in data['image-name'].tolist()])
# saving raw images
np.save('../data/raw_images_flatten.npy', raw_images_flatten)

### applying pca to reduce images' dimensions

In [4]:
def reduce_dim(imgs, k):
    pca = PCA(n_components=k)
    return pca.fit_transform(imgs)

images_600 = reduce_dim(raw_images_flatten, 600)
# saving reduced images
np.save('../data/images_600.npy', images_600)

### using a neural network to create a better mapping

In [5]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Flatten, Dense, concatenate,  Dropout, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
import tensorflow as tf

# embeddings size
EMBED_SIZE = 128

input_image = Input(shape=(128, 128, 4))
x = Conv2D(32, (3, 3), activation='relu')(input_image)
x = MaxPool2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = MaxPool2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu')(x)
x = MaxPool2D((2, 2))(x)
x = Flatten()(x)
x = Dense(EMBED_SIZE, activation='relu')(x)

base_network = Model(inputs=input_image, outputs=x)

def triplet_loss(y_true, y_pred):
    anchor, positive, negative = y_pred[:,:EMBED_SIZE], y_pred[:,EMBED_SIZE:2*EMBED_SIZE], y_pred[:,2*EMBED_SIZE:]
    positive_dist = tf.reduce_mean(tf.square(anchor - positive), axis=1)
    negative_dist = tf.reduce_mean(tf.square(anchor - negative), axis=1)
    return tf.maximum(positive_dist - negative_dist + 0.2, 0.)

input_image_one = Input(shape=(128, 128, 4), name='input_image_one') # input layer for image one
input_image_two = Input(shape=(128, 128, 4), name='input_image_two') # input layer for image two
input_image_three = Input(shape=(128, 128, 4), name='input_image_three') # input layer for image three

out = concatenate([base_network(input_image_one), base_network(input_image_two),
                             base_network(input_image_three)])

model = Model(inputs=[input_image_one, input_image_two, input_image_three],
            outputs=out)
model.compile(loss=triplet_loss, optimizer='rmsprop')

In [6]:
# some utils

def find_positive(df, l):
    
    """
        Find one example of l class.
    """
    
    positive = df[df['species'] == l].sample(1)['image-name'].iloc[0]
    return plt.imread('../data/flowers/'+positive)
    
def find_negative(df, l):
    
    """
        Find one example different from l class.
    """
    
    negative = df[df['species'] != l].sample(1)['image-name'].iloc[0]
    return plt.imread('../data/flowers/'+negative)

In [7]:
# creating a oversized dataset - because there is a random process in the training step
data = data.sample(3500, replace=True)

In [8]:
# creating the training instances
# instance = one image + one similar image (same label) + one different image (different label)

A = []
P = []
N = []

for img, species in zip(data['image-name'].tolist(), data['species'].tolist()):
    
    A.append(plt.imread('../data/flowers/'+img))
    P.append(find_positive(data, species))
    N.append(find_negative(data, species))
    
A = np.array(A)
P = np.array(P)
N = np.array(N)

In [9]:
# training
model.fit([A, P, N], np.zeros((data.shape[0], 3 * EMBED_SIZE)), epochs=10)

Epoch 1/10
110/110 [==============================] - 40s 355ms/step - loss: 0.1237
Epoch 2/10
110/110 [==============================] - 40s 360ms/step - loss: 0.0722
Epoch 3/10
110/110 [==============================] - 37s 340ms/step - loss: 0.0521
Epoch 4/10
110/110 [==============================] - 38s 344ms/step - loss: 0.0312
Epoch 5/10
110/110 [==============================] - 37s 340ms/step - loss: 0.0204
Epoch 6/10
110/110 [==============================] - 37s 335ms/step - loss: 0.0134
Epoch 7/10
110/110 [==============================] - 38s 343ms/step - loss: 0.0101
Epoch 8/10
110/110 [==============================] - 37s 341ms/step - loss: 0.0073
Epoch 9/10
110/110 [==============================] - 38s 349ms/step - loss: 0.0065
Epoch 10/10
110/110 [==============================] - 37s 338ms/step - loss: 0.0054


In [10]:
raw_images = np.array([plt.imread('../data/flowers/'+img) for img in data['image-name'].tolist()])
# creating embeddings for each image
embeddings = base_network.predict(raw_images)
# saving embeddings' images
np.save('../data/embeddings.npy', embeddings)